In [4]:
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten
from tensorflow.keras.layers import Conv2D, MaxPooling2D, AveragePooling2D
from keras import regularizers

from tensorflow.keras.optimizers.schedules import ExponentialDecay
from tensorflow.keras.optimizers import Adam

import numpy as np
import cv2

import os
import random
import pickle
import time

In [2]:
pickle_X_in = open("X.pickle", "rb")
X = pickle.load(pickle_X_in)

pickle_y_in = open("y.pickle", "rb")
y = pickle.load(pickle_y_in)

X = X/255.0

X_train, X_val,y_train, y_val = train_test_split(X, y, test_size=0.1, random_state=241)
y_train = np.array(y_train)
y_val = np.array(y_val)

# Find the best parameters for leraning rate decay

In [12]:
initial_learning_rates = [0.0001, 0.001, 0.01, 0.1]
decay_steps = [500, 1000, 1200, 1500, 1800, 2000]
decay_rates = [0.9, 0.85, 0.8, 0.75, 0.7, 0.65]

for initial_learning_rate in initial_learning_rates:
    for decay_step in decay_steps:
        for decay_rate in decay_rates:
            name = "{}-decay_rate-{}-decay_step-{}-initial_lr-{}".format(decay_rate, decay_step, initial_learning_rate, int(time.time()))
            model = Sequential([
                Conv2D(128, (5, 5), activation="tanh", kernel_regularizer=regularizers.l2(0.01)),
                AveragePooling2D(pool_size=(3, 3)),
                Conv2D(64, (5, 5), activation="tanh", kernel_regularizer=regularizers.l2(0.01)),
                AveragePooling2D(pool_size=(3, 3)),
                Flatten(),
                Dense(64, activation="tanh"),
                Dense(1, activation="sigmoid")
            ])

            learning_rate_schedule = ExponentialDecay(
                initial_learning_rate,
                decay_step,
                decay_rate
            )

            opt = Adam(learning_rate=learning_rate_schedule)

            model.compile(
                loss='binary_crossentropy',
                optimizer=opt,
                metrics = ['accuracy']
            )
            tensorboard = tf.keras.callbacks.TensorBoard(log_dir="lr_logs/{}".format(name))
            history = model.fit(X_train, y_train, batch_size=32, epochs=5, validation_data=(X_val, y_val), shuffle=True,callbacks=[tensorboard])


Epoch 1/5
6/6 [==============================] - 12s 2s/step - loss: 1.5462 - accuracy: 0.5227 - val_loss: 1.4893 - val_accuracy: 0.7000
Epoch 2/5
6/6 [==============================] - 9s 2s/step - loss: 1.4826 - accuracy: 0.6648 - val_loss: 1.4439 - val_accuracy: 0.6000
Epoch 3/5
6/6 [==============================] - 9s 2s/step - loss: 1.4121 - accuracy: 0.7614 - val_loss: 1.3551 - val_accuracy: 0.7500
Epoch 4/5
6/6 [==============================] - 9s 2s/step - loss: 1.3528 - accuracy: 0.7159 - val_loss: 1.2992 - val_accuracy: 0.8000
Epoch 5/5
6/6 [==============================] - 10s 2s/step - loss: 1.3200 - accuracy: 0.7273 - val_loss: 1.2662 - val_accuracy: 0.8000
Epoch 1/5
6/6 [==============================] - 12s 2s/step - loss: 1.5434 - accuracy: 0.5114 - val_loss: 1.4883 - val_accuracy: 0.8000
Epoch 2/5
6/6 [==============================] - 9s 2s/step - loss: 1.4739 - accuracy: 0.7045 - val_loss: 1.4374 - val_accuracy: 0.5500
Epoch 3/5
6/6 [==============================


KeyboardInterrupt



## Conclusion

In [ ]:
'''
After examining the graphs provided by Tensorboard, we find that the model performs best with
the hyper-parameters:
        - initial_learning_rate :    0.8
        - decay_step            :   1500
        - decay rate            :   0.0001

'''